# Example api

In [ ]:
from adelecv.api.logs import enable_logs
from adelecv.api.data.segmentations import SegmentationDataset
from logging import StreamHandler

In [ ]:
enable_logs(StreamHandler())

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# you can change the path to save logs and weights
# from adelecv.api.config import Settings
# from pathlib import Path
# Settings.update_tmp_path(Path('your_path'))

In [ ]:
from adelecv.api.data.segmentations.types import ImageMask

In [ ]:
segm_dataset = SegmentationDataset(
    r'F:\dataset\ph2',
    ImageMask,
    (256, 256),
    (0.7, 0.2, 0.1),
    16,
)

In [ ]:
from adelecv.api.models.segmentations import  get_encoders, get_pretrained_weights, get_models, get_torch_optimizers, get_losses, get_optimize_scores
from adelecv.api.optimize.segmentations import get_hp_optimizers
# создать отдельный модуль для получения возможных параметров

In [ ]:
print('models: ', get_models())
print('encoders: ', get_encoders())
print('pretrained_weights', get_pretrained_weights())
print('optimizers', get_torch_optimizers())
print('losses', get_losses())
print('hp optimizers', get_hp_optimizers())
print('optimize scores', get_optimize_scores())

In [ ]:
from adelecv.api.optimize.segmentations import HyperParamsSegmentation
from adelecv.api.optimize.segmentations import HPOptimizer

In [ ]:
params = HyperParamsSegmentation(
    strategy='TPESampler',
    architectures=['UnetPlusPlus'],
    encoders=['mobilenet_v2'],
    pretrained_weights=['imagenet'],
    loss_fns=['JaccardLoss'],
    optimizers=['AdamW', 'RMSprop'],
    epoch_range=(5, 5),
    lr_range=(0.001, 0.003),
    optimize_score='loss',
)

In [ ]:
hp_optimizer = HPOptimizer(
    hyper_params=params,
    num_trials=1,
    device='GPU',
    dataset=segm_dataset
)

In [ ]:
from adelecv.api.config import Settings

In [ ]:
tensorboard_logs_path = Settings.TENSORBOARD_LOGS_PATH.as_posix()

In [ ]:
tensorboard_logs_path

In [ ]:
%reload_ext tensorboard

In [ ]:
tensorboard_logs_path

In [ ]:
tensorboard --logdir $tensorboard_logs_path

In [ ]:
hp_optimizer.optimize()

In [ ]:
hp_optimizer.stats_models

In [ ]:
from adelecv.api.modification_models import ExportWeights

In [ ]:
# you can save the selected weights in a zip file (feature for ui)
# ExportWeights().create_zip()

In [ ]:
import fiftyone as fo

In [ ]:
fo.launch_app(segm_dataset.fo_dataset)